# Coroutine and thread

## Must-know concept
- Control flow: the order in which individual statements, instructions or function calls are executed or evaluated in a program. 
- Process: 
    - An instance of a computer program that is being executed. 
    - It contains the program code and its current activity. When launching a program, the operating system creates a process for it. 
    - A process has one control flow. When launching multiple threads within a process, the control flow can be multiple.
- Thread: 
    - the smallest sequence of programmed instructions that can be managed independently by a scheduler. 
    - A process can contain multiple threads that share the same memory space.
    - One control flow per thread.
- Coroutine: 
    - a control flow that can span multiple function calls and is cooperatively scheduled within a single thread.
    - take the control flow rotationly by explicitly yielding control back to the caller.

When do we need coroutine?
In the execution, if we encounter an operation that "waits" for a long time, such as I/O operation, network operation, etc., and during this waiting time, the CPU is idle.

### demo: two jobs are interleavedly executed

In [ ]:
import asyncio

async def task_a():
    print("A: step 1")
    await asyncio.sleep(1)   # ← coroutine yields control
    print("A: step 2")

async def task_b():
    print("B: step 1")
    await asyncio.sleep(1)   # ← coroutine yields control
    print("B: step 2")

async def main():
    await asyncio.gather(
        task_a(),
        task_b()
    )

#  asyncio.run() cannot be used in a Jupyter notebook because the kernel already runs an event loop. 
# asyncio.run(main()) --> await main()
await main()


A: step 1
B: step 1
A: step 2
B: step 2
A: step 2
B: step 2


### demo: without `await` 
`bas_task` occupies the control flow and does not yield it, so `good_task` has to wait until `bas_task` is completed.


In [7]:
import asyncio

async def bad_task():
    print("bad start")
    for _ in range(10**8):
        pass
    print("bad end")

async def good_task():
    print("good start")
    await asyncio.sleep(1)
    print("good end")

async def main():
    await asyncio.gather(
        bad_task(),
        good_task()
    )

await main()

bad start
bad end
good start
bad end
good start
good end
good end


### demo: two threads
In multi-threading, the execution order of threads and the output order is out of control of the programmer.

In [2]:
import threading
import time

def task(name):
    print(name, "start")
    time.sleep(1)
    print(name, "end")

t1 = threading.Thread(target=task, args=("A",))
t2 = threading.Thread(target=task, args=("B",))

t1.start()
t2.start()
t1.join()
t2.join()


AB start
 start
B end
A end


In [4]:
!wsl ls -l


total 8
-rwxrwxrwx 1 huhu huhu 4328 Apr 20  2023 cuda-keyring_1.1-1_all.deb
-rwxrwxrwx 1 huhu huhu  286 Jan 16 16:18 hello.cu
